# Implementation - Graphical model

In [1]:
import numpy as np

# 1 Factor graph

### 1.1 Factor-node data structure

### 1.2 Factor-variable data structure

In [2]:
class variable_node:
    def __init__(self, name, neighbors=None):
        self.name = name
        if neighbors is not None:
            self.neighbors = np.array(neighbors)